In [22]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

In [23]:
# Load the data
cosm = pd.read_csv('data/ocosmetic_p.csv')

In [24]:
#All possible combinations for the option choices
option_1 = cosm.Label.unique().tolist()
option_2 = ['Combination', 'Dry', 'Normal', 'Oily', 'Sensitive']
# option_1 = cosm.Label.unique().tolist()
# option_2 = cosm.columns[6:].tolist()

In [26]:
# #filtering the data
df = cosm[cosm['Label'] == 'Moisturizer'][cosm['Combination'] == 1]
df = df.reset_index()
len(df)
# df = cosm[cosm['Label'] == option_1][cosm[option_2] == 1]
# df = df.reset_index()
# len(df)

<ipython-input-26-b1c1249f5bdd>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = cosm[cosm['Label'] == 'Moisturizer'][cosm['Combination'] == 1]


199

In [27]:

    
    #tokenization
    word_index_map = {}
    index_word_map = []
    current_index = 0
    corpus = []

    for i in range(len(df)):
        text = df['ingredients'][i]
        text = text.lower()
        tokens = text.split(', ')
        corpus.append(tokens)
        for ingredient in tokens:
            if ingredient not in word_index_map:
                word_index_map[ingredient] = current_index
                current_index += 1
                index_word_map.append(ingredient)
                
    # prepare matrix using numpy
    D = len(corpus)                 # The number of documents
    N = len(word_index_map)     # The number of the tokens

    # Initialize a matrix of zeros
    X = np.zeros((D, N))

    #vectorizing the tokens
    def tokens_to_vector(tokens):
        #initializing empty array
        x = np.zeros(N)
        for t in tokens:
            # Get the index for each ingredient
            i = word_index_map[t]
            # Put 1 at the corresponding indices
            x[i] = 1
        return x

    # Make a document-term matrix
    i = 0
    for tokens in corpus:
        X[i, :] = tokens_to_vector(tokens)
        i += 1

    # Dimension reduction with SVD
    svd = TruncatedSVD()
    Z = svd.fit_transform(X)
    Z_df = pd.DataFrame(data = Z, columns = ['SVD1', 'SVD2'])   
    cosm_svd = pd.concat([df.iloc[:, 1:6], Z_df], axis = 1)
    cosm_svd.tail()


,Label,brand,name,price,rank,SVD1,SVD2
194,Moisturizer,SHISEIDO,White Lucent All Day Brightener Broad Spectrum...,62,4.6,2.933927,-0.166521
195,Moisturizer,SATURDAY SKIN,Featherweight Daily Moisturizing Cream,49,4.6,3.069613,-1.523037
196,Moisturizer,HERBIVORE,Jasmine Green Tea Balancing Toner,39,4.2,0.082719,-0.046447
197,Moisturizer,KATE SOMERVILLE,Goat Milk Moisturizing Cream,65,4.1,2.531714,-0.327133
198,Moisturizer,GO-TO,Face Hero,34,4.8,0.000019,0.000163


In [28]:
cosm_svd.tail()

,Label,brand,name,price,rank,SVD1,SVD2
194,Moisturizer,SHISEIDO,White Lucent All Day Brightener Broad Spectrum...,62,4.6,2.933927,-0.166521
195,Moisturizer,SATURDAY SKIN,Featherweight Daily Moisturizing Cream,49,4.6,3.069613,-1.523037
196,Moisturizer,HERBIVORE,Jasmine Green Tea Balancing Toner,39,4.2,0.082719,-0.046447
197,Moisturizer,KATE SOMERVILLE,Goat Milk Moisturizing Cream,65,4.1,2.531714,-0.327133
198,Moisturizer,GO-TO,Face Hero,34,4.8,0.000019,0.000163


In [7]:
# Save the file
cosm_svd.to_csv('data/skincare_SVD12.csv', encoding = 'utf-8-sig', index = False)

In [8]:
cosm_svd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Label   190 non-null    object 
 1   brand   190 non-null    object 
 2   name    190 non-null    object 
 3   price   190 non-null    int64  
 4   rank    190 non-null    float64
 5   SVD1    190 non-null    float64
 6   SVD2    190 non-null    float64
dtypes: float64(3), int64(1), object(3)
memory usage: 8.2+ KB


In [9]:
cosm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        1472 non-null   object 
 1   brand        1472 non-null   object 
 2   name         1472 non-null   object 
 3   price        1472 non-null   int64  
 4   rank         1472 non-null   float64
 5   ingredients  1472 non-null   object 
 6   Combination  1472 non-null   int64  
 7   Dry          1472 non-null   int64  
 8   Normal       1472 non-null   int64  
 9   Oily         1472 non-null   int64  
 10  Sensitive    1472 non-null   int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 103.6+ KB
